In [1]:
# для работы с локальной базой данных воспользуемся sqlite3, импортируем её
# создаём подключение к базе данных, если такое название отсутствует в корневой папке, файл автоматически будет создан
import sqlite3
conn = sqlite3.connect('beorg.db')
c = conn.cursor()

In [2]:
# создаём для нашей базы первую таблицу clients

# код комментируем, чтобы весь проект можно было заново запускать без ошибок


# c.execute("""CREATE TABLE clients (
#     client_id integer,
#     date_from text,
#     date_to text)""")
          
# conn.commit()

In [3]:
# первая таблица совсем маленькая, бытсрей будет просто ввести нужные данные несколькими запросами


# c.execute("INSERT INTO clients VALUES (1, '01.01.2014','31.12.9999')")
# c.execute("INSERT INTO clients VALUES (2, '01.02.2015','31.12.9999')")
# c.execute("INSERT INTO clients VALUES (3, '01.02.2015','01.01.2016')")
# conn.commit()

In [4]:
# для работы с таблицами воспользуемся pandas
# таблицы у нас представлены в файле excel
import pandas as pd

In [5]:
# получим эту таблицу и выведим на экран
products = pd.read_excel('sql.xlsx', sheet_name = 'products')
products

,ProductType,ClientID,Дата открытия,Дата закрытия,Офис обслуживания
0,Услуга_1,1,2014-08-29,4444-01-01 00:00:00,ДО 1
1,Услуга_1,1,2015-06-16,4444-01-01 00:00:00,ДО 2
2,Услуга_1,2,2014-03-04,2014-04-22 00:00:00,ДО 1
3,Услуга_2,1,2014-02-26,4444-01-01 00:00:00,ДО 2
4,Услуга_2,3,2014-12-11,2016-01-01 00:00:00,ДО 3
5,Услуга_2,1,2015-08-11,2015-11-11 00:00:00,ДО 1
6,Услуга_2,2,2014-08-29,2014-10-21 00:00:00,ДО 5
7,Услуга_3,1,2014-07-23,4444-01-01 00:00:00,ДО 4
8,Услуга_4,3,2014-03-03,4444-01-01 00:00:00,ДО 1
9,Услуга_5,2,2014-03-04,4444-01-01 00:00:00,ДО 3


In [6]:
# создадим вторую таблицу products

# код также комментируем для возможности снова запускать весь код


# c.execute("""CREATE TABLE products (
#     product_type text,
#     client_id integer,
#     date_start text,
#     date_close text,
#     office text)""")
          
# conn.commit()

In [7]:
# для экспорта данных в таблицу в нашей базе данных обратимся к каждой строке импортированной с помощью pandas таблицы



# for x in range(0,11):
#     c.execute("INSERT INTO products VALUES ('{}', {}, '{}','{}','{}')".format(products.loc[x][0],products.loc[x][1],
#                                                                products.loc[x][2],products.loc[x][3],
#                                                               products.loc[x][4]))
    
    
# conn.commit()

In [8]:
# теперь обе таблицы есть и можно выполнять запросы к ним в sqlite3

In [9]:
# 1. Вывести уникальные ID клиентов, имеющих действующий продукт "Услуга_1"
# выводим уникальные значения id клиентов, для которых услуга_1 ещё активна, 
# значит она есть и дата окончания больше текущей даты
c.execute("""SELECT
                DISTINCT client_id
            FROM
                products
            WHERE 
                product_type == 'Услуга_1' AND
                date_close > '2021-01-01'
          """)
c.fetchall()

[(1,)]

In [10]:
# 2. Вывести наименование и количество действующих продуктов
# выводим тип продукта и его количество
# в условии указываем проверку активности, а группировкой находим необходимое количество для каждого продукта
c.execute("""SELECT
            product_type,
            COUNT(product_type) as cnt
            FROM
            products
            WHERE date_close > '2021-01-01'
            GROUP BY
            product_type
          """)
c.fetchall()

[('Услуга_1', 2),
 ('Услуга_2', 1),
 ('Услуга_3', 1),
 ('Услуга_4', 1),
 ('Услуга_5', 1)]

In [11]:
# 3. Вывести уникальные ID действующих клиентов, имеющих действующий продукт "Услуга_2"
# выводим уникальные id клиента, берём это из объединённой таблицы, 
# задаём условия, в таблице с клиентами берём только активных, в таблице с продуктами также фильтруем по активности и
# берём только услуга_2
c.execute("""SELECT
                DISTINCT clients.client_id
            FROM
                products
            INNER JOIN clients ON clients.client_id = products.client_id
            WHERE
                clients.date_to > '01.01.2021' AND
                products.date_close > '2021-01-01' AND
                products.product_type == 'Услуга_2'
          """)
c.fetchall()

[(1,)]

In [12]:
# 4. Вывести уникальные ID действующих клиентов, у которых отсутствует услуга "Услуга_3"
# для того чтобы найти клиентов без услуги_3, найдём тех у кого она есть
# в условии указываем, что нам нужны клиенты, все кроме только что найденных клиентов с услугой_3
# отфильтрованную таблицу products объединяем с таблицей с клиентами и находим уникальных клиентов
# а в условии говорим, что клиент должен быть активным
c.execute("""SELECT
                DISTINCT clients.client_id
            FROM
                (SELECT 
                    DISTINCT client_id AS client_id
                FROM
                    products
                WHERE 
                client_id != (SELECT 
                                client_id 
                            FROM 
                                products
                            WHERE
                                product_type == 'Услуга_3')) as service_3
            INNER JOIN clients ON clients.client_id = service_3.client_id
            WHERE
                clients.date_to > '01.01.2021'

          """)
c.fetchall()

[(2,)]

In [13]:
# 5. Для таблицы Products добавить флаг активности продукта (1 - активен, 0 - закрыт)



# для добавления нового столбца в таблицу с текущим названием нужно переименовать таблицу
# так мы сможем создать новую таблицу используя старое название
# c.execute("""ALTER TABLE products RENAME TO products_old""")
          
# conn.commit()

In [14]:
# создаём новую таблицу со старым названием, теперь она пустая и содержит новый столбец



# c.execute("""CREATE TABLE products (
#     product_type text,
#     client_id integer,
#     date_start text,
#     date_close text,
#     office text,
#     active integer)""")
          
# conn.commit()

In [15]:
# вставляем в новую таблицу все значения старой таблицы и в конце указываем как заполнять новый столбец
# в нашем случае это CASE, для активных 1, для неактивных 0


In [16]:
# проверка наличия нового столбца и значений в нём для таблицы
c.execute("""SELECT 
    *
    FROM 
        products
        """)

c.fetchall()

[('Услуга_1', 1, '2014-08-29 00:00:00', '4444-01-01 00:00:00', 'ДО 1', 1),
 ('Услуга_1', 1, '2015-06-16 00:00:00', '4444-01-01 00:00:00', 'ДО 2', 1),
 ('Услуга_1', 2, '2014-03-04 00:00:00', '2014-04-22 00:00:00', 'ДО 1', 0),
 ('Услуга_2', 1, '2014-02-26 00:00:00', '4444-01-01 00:00:00', 'ДО 2', 1),
 ('Услуга_2', 3, '2014-12-11 00:00:00', '2016-01-01 00:00:00', 'ДО 3', 0),
 ('Услуга_2', 1, '2015-08-11 00:00:00', '2015-11-11 00:00:00', 'ДО 1', 0),
 ('Услуга_2', 2, '2014-08-29 00:00:00', '2014-10-21 00:00:00', 'ДО 5', 0),
 ('Услуга_3', 1, '2014-07-23 00:00:00', '4444-01-01 00:00:00', 'ДО 4', 1),
 ('Услуга_4', 3, '2014-03-03 00:00:00', '4444-01-01 00:00:00', 'ДО 1', 1),
 ('Услуга_5', 2, '2014-03-04 00:00:00', '4444-01-01 00:00:00', 'ДО 3', 1),
 ('Услуга_6', 1, '2014-12-16 00:00:00', '2015-04-21 00:00:00', 'ДО 1', 0)]